In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
df=pd.read_csv("../input/digit-recognizer/train.csv")
df.head()

In [ ]:
df_test=pd.read_csv("../input/digit-recognizer/test.csv").values
df_test[0:5]

In [ ]:
X=df.drop(["label"],axis="columns").values
Y=df["label"].values
X=X/255.0

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

In [ ]:
X=X.reshape(len(X),28,28,-1)
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

# Data Augumentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=15,  
        zoom_range = 0.15,  
        width_shift_range=0.1,  
        height_shift_range=0.1, 
        horizontal_flip=False,  
        vertical_flip=False) 

datagen.fit(x_train)

In [ ]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16,(3,3),padding='same',activation='relu',input_shape=(28,28,1)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv2D(32,(3,3),padding='same',activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(10, activation="softmax")
    ])




# optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
model.summary()    

In [ ]:
batch_size = 64
epochs = 30
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size), epochs = epochs, 
                              validation_data = (x_test, y_test), verbose=2, 
                              steps_per_epoch=x_train.shape[0] // batch_size,
                              callbacks=[reduce_lr])

In [ ]:
model.save('model.h5')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train','Test'])
plt.show()

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred=model.predict(x_test)
y_preds=[np.argmax(pred) for pred in y_pred]
accuracy_score(y_preds,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_mtx = confusion_matrix(y_test,y_preds) 
confusion_mtx

In [ ]:
df_test=df_test.reshape(len(df_test),28,28,-1)
a_pred=model.predict((df_test)/255.0)
a_preds=[np.argmax(pred) for pred in a_pred]
    
samp=pd.read_csv("../input/digit-recognizer/sample_submission.csv")
samp["Label"]=a_preds
samp.head()

In [ ]:
samp.to_csv("outputCNN.csv",index=False)